In [ ]:
# Install library yang diperlukan (jika belum terinstall)
!pip install transformers scikit-learn pandas torch tqdm

import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

# Setup device (gunakan GPU jika tersedia)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

Menggunakan device: cuda


In [ ]:
# URL Dataset
train_url = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/refs/heads/main/data/incidents_train.csv"
valid_url = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/refs/heads/main/data/incidents_valid.csv"
test_url = "https://raw.githubusercontent.com/food-hazard-detection-semeval-2025/food-hazard-detection-semeval-2025.github.io/refs/heads/main/data/incidents_test.csv"

# Load Dataframes
df_train = pd.read_csv(train_url)
df_valid = pd.read_csv(valid_url)
df_test = pd.read_csv(test_url)

# Tampilkan sampel data untuk memastikan kolom yang benar
print(f"Jumlah Data Train: {len(df_train)}")
print(f"Jumlah Data Valid: {len(df_valid)}")
print(f"Jumlah Data Test: {len(df_test)}")

print("\nContoh Data Train:")
display(df_train.head(3))

# PENTING: Menangani Missing Values
# Kita mengisi nilai NaN dengan string kosong agar tokenizer tidak error
df_train = df_train.fillna('')
df_valid = df_valid.fillna('')
df_test = df_test.fillna('')

Jumlah Data Train: 5082
Jumlah Data Valid: 565
Jumlah Data Test: 997

Contoh Data Train:


,Unnamed: 0,year,month,day,country,title,text,hazard-category,product-category,hazard,product
0,0,1994,1,7,us,Recall Notification: FSIS-024-94,Case Number: 024-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,smoked sausage
1,1,1994,3,10,us,Recall Notification: FSIS-033-94,Case Number: 033-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria spp,sausage
2,2,1994,3,28,us,Recall Notification: FSIS-014-94,Case Number: 014-94 \n Date Opene...,biological,"meat, egg and dairy products",listeria monocytogenes,ham slices


In [ ]:
# Inisialisasi Encoder
hazard_encoder = LabelEncoder()
product_encoder = LabelEncoder()

# Kita fit encoder menggunakan gabungan seluruh data (Train+Valid+Test)
# untuk memastikan semua kemungkinan label tercover.
all_hazards = pd.concat([df_train['hazard-category'], df_valid['hazard-category'], df_test['hazard-category']]).unique()
all_products = pd.concat([df_train['product-category'], df_valid['product-category'], df_test['product-category']]).unique()

hazard_encoder.fit(all_hazards)
product_encoder.fit(all_products)

# Transformasi label menjadi angka
def encode_labels(df):
    df['hazard_label'] = hazard_encoder.transform(df['hazard-category'])
    df['product_label'] = product_encoder.transform(df['product-category'])
    return df

df_train = encode_labels(df_train)
df_valid = encode_labels(df_valid)
df_test = encode_labels(df_test)

num_hazard_classes = len(hazard_encoder.classes_)
num_product_classes = len(product_encoder.classes_)

print(f"Jumlah Kelas Hazard: {num_hazard_classes}")
print(f"Jumlah Kelas Product: {num_product_classes}")

Jumlah Kelas Hazard: 10
Jumlah Kelas Product: 22


In [ ]:
# Inisialisasi Tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
MAX_LEN = 128  # Panjang maksimal token (bisa dinaikkan ke 256/512 jika memori cukup)

class FoodHazardDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Mengambil teks judul
        title = str(self.data.iloc[index]['title'])
        # Opsional: Jika ingin menggabungkan teks dengan metadata negara/tanggal, bisa dilakukan di sini
        # contoh: text = title + " [SEP] " + str(self.data.iloc[index]['country'])

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'hazard_label': torch.tensor(self.data.iloc[index]['hazard_label'], dtype=torch.long),
            'product_label': torch.tensor(self.data.iloc[index]['product_label'], dtype=torch.long)
        }

# Membuat DataLoader
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16

train_dataset = FoodHazardDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = FoodHazardDataset(df_valid, tokenizer, MAX_LEN)
test_dataset = FoodHazardDataset(df_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class MultiTaskBERT(nn.Module):
    def __init__(self, num_hazard_classes, num_product_classes):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(0.3)

        # Head 1: Klasifikasi Hazard
        self.hazard_out = nn.Linear(768, num_hazard_classes)

        # Head 2: Klasifikasi Product
        self.product_out = nn.Linear(768, num_product_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        # Forward pass melalui BERT
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # Mengambil output dari token [CLS] (pooled_output)
        pooled_output = outputs[1]
        output = self.drop(pooled_output)

        # Prediksi untuk masing-masing task
        hazard_logits = self.hazard_out(output)
        product_logits = self.product_out(output)

        return hazard_logits, product_logits

model = MultiTaskBERT(num_hazard_classes, num_product_classes)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

MultiTaskBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
EPOCHS = 4

# PERBAIKAN DI SINI: Hapus parameter 'correct_bias=False'
optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Loss function
criterion = nn.CrossEntropyLoss()

def train_epoch(model, data_loader, optimizer, device, scheduler):
    model = model.train()
    total_loss = 0

    for d in tqdm(data_loader, desc="Training"):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        token_type_ids = d["token_type_ids"].to(device)
        hazard_labels = d["hazard_label"].to(device)
        product_labels = d["product_label"].to(device)

        optimizer.zero_grad()

        # Forward Pass
        h_logits, p_logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # Hitung Loss (Multi-task Loss: Loss Hazard + Loss Product)
        loss_h = criterion(h_logits, hazard_labels)
        loss_p = criterion(p_logits, product_labels)
        loss = loss_h + loss_p

        total_loss += loss.item()

        # Backward Pass
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    return total_loss / len(data_loader)

def eval_model(model, data_loader, device):
    model = model.eval()

    # Simpan prediksi dan label asli
    h_preds, h_true = [], []
    p_preds, p_true = [], []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            token_type_ids = d["token_type_ids"].to(device)

            h_logits, p_logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            # Ambil prediksi dengan probabilitas tertinggi
            _, h_predicted = torch.max(h_logits, dim=1)
            _, p_predicted = torch.max(p_logits, dim=1)

            h_preds.extend(h_predicted.cpu().numpy())
            h_true.extend(d["hazard_label"].cpu().numpy())

            p_preds.extend(p_predicted.cpu().numpy())
            p_true.extend(d["product_label"].cpu().numpy())

    return h_true, h_preds, p_true, p_preds

In [ ]:
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)

    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    print(f"Train Loss: {train_loss:.4f}")

    # Validasi
    h_true, h_preds, p_true, p_preds = eval_model(model, valid_loader, device)

    # Hitung Macro F1 Score
    f1_hazard = f1_score(h_true, h_preds, average='macro')
    f1_product = f1_score(p_true, p_preds, average='macro')

    # Rata-rata Macro F1 dari kedua task (sebagai metrik gabungan)
    avg_f1 = (f1_hazard + f1_product) / 2

    print(f"Val Hazard Macro F1 : {f1_hazard:.4f}")
    print(f"Val Product Macro F1: {f1_product:.4f}")
    print(f"Combined Macro F1   : {avg_f1:.4f}")


Epoch 1/4
----------


Training: 100%|██████████| 318/318 [01:42<00:00,  3.09it/s]


Train Loss: 3.5338
Val Hazard Macro F1 : 0.4250
Val Product Macro F1: 0.1864
Combined Macro F1   : 0.3057

Epoch 2/4
----------


Training: 100%|██████████| 318/318 [01:49<00:00,  2.90it/s]


Train Loss: 2.3199
Val Hazard Macro F1 : 0.4519
Val Product Macro F1: 0.3521
Combined Macro F1   : 0.4020

Epoch 3/4
----------


Training: 100%|██████████| 318/318 [01:49<00:00,  2.91it/s]


Train Loss: 1.8293
Val Hazard Macro F1 : 0.4993
Val Product Macro F1: 0.4246
Combined Macro F1   : 0.4620

Epoch 4/4
----------


Training: 100%|██████████| 318/318 [01:49<00:00,  2.91it/s]


Train Loss: 1.5851
Val Hazard Macro F1 : 0.5424
Val Product Macro F1: 0.4337
Combined Macro F1   : 0.4881


In [ ]:
print("\n--- Final Evaluation on Test Set ---")
h_true_test, h_preds_test, p_true_test, p_preds_test = eval_model(model, test_loader, device)

# --- 3. Hitung Metrik ---
f1_hazard_test = f1_score(h_true_test, h_preds_test, average='macro')
f1_product_test = f1_score(p_true_test, p_preds_test, average='macro')
avg_f1_test = (f1_hazard_test + f1_product_test) / 2

print(f"Test Hazard Macro F1 : {f1_hazard_test:.4f}")
print(f"Test Product Macro F1: {f1_product_test:.4f}")
print(f"Combined Macro F1    : {avg_f1_test:.4f}")

# --- 4. Tampilkan Hasil dalam Tabel (DataFrame) ---
# Konversi angka ke teks
pred_hazard_text = hazard_encoder.inverse_transform(h_preds_test)
pred_product_text = product_encoder.inverse_transform(p_preds_test)

# Buat DataFrame hasil
df_results = df_test.copy()
df_results['pred_hazard'] = pred_hazard_text
df_results['pred_product'] = pred_product_text

# Cek kebenaran prediksi
df_results['hazard_correct'] = df_results['hazard-category'] == df_results['pred_hazard']
df_results['product_correct'] = df_results['product-category'] == df_results['pred_product']

# Atur tampilan pandas agar rapi
pd.set_option('display.max_colwidth', 100)

cols_to_show = ['title', 'hazard-category', 'pred_hazard', 'product-category', 'pred_product']

print(f"\n=== Menampilkan 20 Data Pertama ===")
display(df_results[cols_to_show].head(20))


--- Final Evaluation on Test Set ---
Test Hazard Macro F1 : 0.4929
Test Product Macro F1: 0.3912
Combined Macro F1    : 0.4421

=== Menampilkan 20 Data Pertama ===


,title,hazard-category,pred_hazard,product-category,pred_product
0,Recall Notification: FSIS-039-94,biological,biological,"meat, egg and dairy products","meat, egg and dairy products"
1,Recall Notification: FSIS-026-95,biological,biological,fruits and vegetables,"meat, egg and dairy products"
2,Recall Notification: FSIS-028-95,biological,biological,"meat, egg and dairy products","meat, egg and dairy products"
3,Black & Gold—Sliced Silverside 250g,biological,allergens,"meat, egg and dairy products","meat, egg and dairy products"
4,Nestle—Peters—Neapolitan Ice Cream,foreign bodies,allergens,ices and desserts,ices and desserts
5,Knispel Fruit Juices—Chilled Fruit Juice,biological,foreign bodies,non-alcoholic beverages,non-alcoholic beverages
6,Woolworths Limited—Cheese King baked ricotta,biological,biological,"meat, egg and dairy products","meat, egg and dairy products"
7,West Coast Halal Meat Specialists—Halal Sliced Polony,biological,biological,"meat, egg and dairy products","meat, egg and dairy products"
8,Leggos—Stir Through pasta sauces,foreign bodies,allergens,"soups, broths, sauces and condiments","soups, broths, sauces and condiments"
9,Brisbane Fish Markets—Queenfish,chemical,biological,seafood,seafood
